In [4]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests




In [5]:
cph = pd.read_csv("copenhagen_imputed.csv")
osl = pd.read_csv("oslo_imputed.csv")

In [6]:
def bh_correlations(
    df,
    drop_cols=None,
    alpha=0.05,
    min_effect=0.8,
    method="pearson"
):
    """
    Returns BH-significant correlations with large effect size.
    H0: rho_ij = 0
    """
    if drop_cols is None:
        drop_cols = []

    X = (
        df.select_dtypes(include="number")
          .drop(columns=drop_cols, errors="ignore")
    )

    results = []

    for f1, f2 in combinations(X.columns, 2):
        r, p = pearsonr(X[f1], X[f2])
        results.append({
            "feature_1": f1,
            "feature_2": f2,
            "r": r,
            "p_value": p
        })

    res = pd.DataFrame(results)

    reject, p_bh, _, _ = multipletests(
        res["p_value"],
        alpha=alpha,
        method="fdr_bh"
    )

    res["p_value_bh"] = p_bh
    res["reject_bh"] = reject

    # keep only statistically + practically meaningful
    res = res[
        (res["reject_bh"]) &
        (res["r"].abs() >= min_effect)
    ].copy()

    res["abs_r"] = res["r"].abs()
    res = res.sort_values("abs_r", ascending=False)

    return res

In [9]:

DROP_COLS = ["Unnamed: 0"]

cph_sig = bh_correlations(
    cph,
    drop_cols=DROP_COLS,
    alpha=0.05,
    min_effect=0.8
)

osl_sig = bh_correlations(
    osl,
    drop_cols=DROP_COLS,
    alpha=0.05,
    min_effect=0.8
)

print("Copenhagen strong BH-significant correlations:")
print(cph_sig.head(10))

print("\nOslo strong BH-significant correlations:")
print(osl_sig.head(10))

Copenhagen strong BH-significant correlations:
                          feature_1  \
726  calculated_host_listings_count   
391          maximum_maximum_nights   
365          minimum_maximum_nights   
337          maximum_minimum_nights   
0               host_listings_count   
488                 availability_60   
465                 availability_30   
363          minimum_maximum_nights   
70        host_total_listings_count   
69        host_total_listings_count   

                                       feature_2         r  p_value  \
726  calculated_host_listings_count_entire_homes  0.999390      0.0   
391                       maximum_nights_avg_ntm  0.981362      0.0   
365                       maximum_nights_avg_ntm  0.973890      0.0   
337                       minimum_nights_avg_ntm  0.960574      0.0   
0                      host_total_listings_count  0.954141      0.0   
488                              availability_90  0.948037      0.0   
465                       

In [8]:
def canonical_pair(df):
    return df.apply(
        lambda r: tuple(sorted([r["feature_1"], r["feature_2"]])),
        axis=1
    )

cph_pairs = set(canonical_pair(cph_sig))
osl_pairs = set(canonical_pair(osl_sig))

common_pairs = cph_pairs & osl_pairs
cph_only = cph_pairs - osl_pairs
osl_only = osl_pairs - cph_pairs

print(f"Common strong correlations: {len(common_pairs)}")
print(f"Copenhagen-only: {len(cph_only)}")
print(f"Oslo-only: {len(osl_only)}")


Common strong correlations: 11
Copenhagen-only: 7
Oslo-only: 9


In [ ]:
from collections import defaultdict, deque

def print_collapse_features(sig_df, title=""):
    """
    Given a dataframe of significant correlations, prints clusters of features
    that should be collapsed.
    Expects columns: feature_1, feature_2, r, p_value_bh (optional).
    """
    if sig_df is None or sig_df.empty:
        print(f"\n{title}\nNo collapse candidates (no strong BH-significant correlations).")
        return

    # Build undirected graph
    graph = defaultdict(set)
    edges = {}
    for _, row in sig_df.iterrows():
        f1, f2 = row["feature_1"], row["feature_2"]
        graph[f1].add(f2)
        graph[f2].add(f1)

        # store edge evidence (use canonical key)
        key = tuple(sorted((f1, f2)))
        edges[key] = {
            "r": float(row["r"]),
            "p_bh": float(row["p_value_bh"]) if "p_value_bh" in sig_df.columns else None
        }

    # Find connected components
    visited = set()
    clusters = []

    for node in graph:
        if node in visited:
            continue
        q = deque([node])
        visited.add(node)
        comp = {node}

        while q:
            cur = q.popleft()
            for nb in graph[cur]:
                if nb not in visited:
                    visited.add(nb)
                    comp.add(nb)
                    q.append(nb)

        clusters.append(comp)


    print(f"\n{title}")
    print("=" * len(title))

    # Sort clusters by size then name
    clusters = sorted(clusters, key=lambda s: (-len(s), sorted(s)[0]))

    for i, cluster in enumerate(clusters, 1):
        cluster_list = sorted(cluster)
        print(f"\nCluster {i} (size={len(cluster_list)}):")
        for f in cluster_list:
            print(f"  - {f}")

        # Print evidence edges inside the cluster
        print("  Evidence (pairs inside cluster):")
        cluster_set = set(cluster_list)
        inner_keys = [
            k for k in edges.keys()
            if k[0] in cluster_set and k[1] in cluster_set
        ]

        # sort by |r| desc
        inner_keys = sorted(inner_keys, key=lambda k: abs(edges[k]["r"]), reverse=True)

        for k in inner_keys:
            info = edges[k]
            if info["p_bh"] is None:
                print(f"    {k[0]} <-> {k[1]}  (r={info['r']:.3f})")
            else:
                print(f"    {k[0]} <-> {k[1]}  (r={info['r']:.3f}, p_bh={info['p_bh']:.2e})")


print_collapse_features(cph_sig, title="Copenhagen collapse candidates")
print_collapse_features(osl_sig, title="Oslo collapse candidates")



Copenhagen collapse candidates

Cluster 1 (size=5):
  - availability_30
  - availability_365
  - availability_60
  - availability_90
  - availability_eoy
  Evidence (pairs inside cluster):
    availability_60 <-> availability_90  (r=0.948, p_bh=0.00e+00)
    availability_30 <-> availability_60  (r=0.919, p_bh=0.00e+00)
    availability_30 <-> availability_90  (r=0.834, p_bh=0.00e+00)
    availability_90 <-> availability_eoy  (r=0.818, p_bh=0.00e+00)
    availability_365 <-> availability_eoy  (r=0.815, p_bh=0.00e+00)

Cluster 2 (size=4):
  - calculated_host_listings_count
  - calculated_host_listings_count_entire_homes
  - host_listings_count
  - host_total_listings_count
  Evidence (pairs inside cluster):
    calculated_host_listings_count <-> calculated_host_listings_count_entire_homes  (r=0.999, p_bh=0.00e+00)
    host_listings_count <-> host_total_listings_count  (r=0.954, p_bh=0.00e+00)
    calculated_host_listings_count_entire_homes <-> host_total_listings_count  (r=0.894, p_bh=0